## Bootloaders


## Grub 2

![img](img/bl/grub2.png)

### Configuration

In [19]:
ls /dev/sda* --color=no

/dev/sda  /dev/sda1  /dev/sda2  /dev/sda5


In [ ]:
# Install on MBR
grub-install /dev/sda

In [29]:
# Defaul config fike
grep -E '^#?GRUB.+' --color=no /etc/default/grub 

GRUB_DEFAULT=0
#GRUB_HIDDEN_TIMEOUT=0
#GRUB_HIDDEN_TIMEOUT_QUIET=true
GRUB_TIMEOUT=10
GRUB_DISTRIBUTOR=`lsb_release -i -s 2> /dev/null || echo Debian`
GRUB_CMDLINE_LINUX_DEFAULT=""
#GRUB_CMDLINE_LINUX_DEFAULT="quiet splash"
GRUB_CMDLINE_LINUX=""
#GRUB_BADRAM="0x01234567,0xfefefefe,0x89abcdef,0xefefefef"
#GRUB_TERMINAL=console
#GRUB_GFXMODE=640x480
#GRUB_DISABLE_LINUX_UUID=true
#GRUB_DISABLE_RECOVERY="true"
#GRUB_INIT_TUNE="480 440 1"


- `GRUB_DEFAULT`
    - The default menu entry. This may be a number or the special string `saved` (`sudo grub-set-default`).  
    - "2>2" submenus
    
- `GRUB_SAVEDEFAULT`
    - If this option is set to `true`, then, when an entry is selected, save it as a new default entry for use by future runs of GRUB.  


- `GRUB_TIMEOUT`
    - Boot the default entry this many seconds after the menu is displayed.  
    - Set to `0` to boot immediately without displaying the menu, or to `-1` to wait indefinitely.  


- `GRUB_HIDDEN_TIMEOUT`    
    - Wait this many seconds before displaying the menu. If ESC is pressed during that time, display the menu.  
    - If you set `GRUB_HIDDEN_TIMEOUT`, you should also set `GRUB_TIMEOUT=0` so that the menu is not displayed at all unless ESC is pressed.  


- `GRUB_HIDDEN_TIMEOUT_QUIET`
    - to suppress the verbose countdown while waiting for a key to be pressed before displaying the menu.  


- `GRUB_DISTRIBUTOR`
    - Set by distributors of GRUB to their identifying name. This is used to generate more informative menu entry titles.  


- `GRUB_CMDLINE_LINUX_DEFAULT`
    - Unless `GRUB_DISABLE_RECOVERY` is set to `true`, two menu entries will be generated for each Linux kernel: one default entry and one entry for recovery mode. This option lists command-line arguments to add only to the **default** menu entry, after those listed in 
`GRUB_CMDLINE_LINUX`.  


- `GRUB_CMDLINE_LINUX`
    - Command-line arguments to add to menu entries for the Linux kernel.  
    
    
- `GRUB_RECORDFAIL_TIMEOUT`
    - Show menu if last boot has been failed
        - `-1 wait`
        -  `0 do not wait no matter what`
        - `>0 wait x second`
        
- `GRUB_DISABLE_SUBMENU`
    - `y`

In [33]:
lsb_release -i  -s || echo debian

Ubuntu


In [ ]:
grub-mkconfig -o /boot/grub/grub.cfg # /boot/grub2/grub.cfg in some: grub.conf

In [ ]:
update-grub

In [1]:
whatis update-grub

update-grub (8)      - stub for grub-mkconfig


In [4]:
head -5 /boot/grub/grub.cfg

#
# DO NOT EDIT THIS FILE
#
# It is automatically generated by grub-mkconfig using templates
# from /etc/grub.d and settings from /etc/default/grub


grub.cfg is overwritten by certain Grub 2 package updates, whenever a kernel is added or removed, or when the user runs update-grub. 

In [2]:
ls /etc/grub.d

00_header        10_linux      30_os-prober      40_custom  README
05_debian_theme  20_linux_xen  30_uefi-firmware  41_custom


In [31]:
grep "*" /etc/grub.d/README

  00_*: Reserved for 00_header.
  10_*: Native boot entries.
  20_*: Third party apps (e.g. memtest86+).


### Password protected entries

<span style="color:darkred">Persons with physical access to the computer can gain access to the files via other methods which GRUB 2 cannot prevent. </span>

In [ ]:
# What people do
sudo nano /etc/grub.d/00_header
sudo nano /etc/grub.d/01_users

# Should use cat << END ... END

In [ ]:
# What I do
sudo cp /etc/grub.d/40_custom /etc/grub.d/42_custom

#### Superuser
Can select/edit anything and have access to terminal.

In [ ]:
set superusers="ravexina"
password ravexina 123

# -- other users optional -- #
password milad 111
password userb 222

#### Secure password

In [ ]:
grub-mkpasswd-pbkdf2

    Enter password: 
    Reenter password: 
    PBKDF2 hash of your password is grub.pbkdf2.sha512.10000.048CFA9270E0BC8ECF22A27100538B0864C224E50E9179E018CBDF16FEB788C5D8510158BA65195885E5DB1D2A634C0AAEDF5268A1BF519C64BEB0E3DAF9B208.8E7D8ECC5662FA2431AF078B400788EF93937A324D92B0BC71400554D3E5949092F8CD7851995300479C71BC31E820C43065993CD395DAE4A1FAFC681DB1C668

`set superusers="milad"`  
`password_pbkdf2 milad grub.pbkdf2.sha512.10000.048CFA9270E0BC8ECF22A27100538B0864C224E5...`

#### Protecting menu entries

In [ ]:
menuentry --class os "May be run by any user" --unrestricted {

menuentry --class os "Superusers only" --users "" {

menuentry --class os "May be run by user1 and user2 or a superuser" --users user1,user2 {

In [48]:
# to protect them all
grep  "echo \"menuentry" /etc/grub.d/10_linux

      echo "menuentry '$(echo "$title" | grub_quote)' ${CLASS} \$menuentry_id_option 'gnulinux-$version-$type-$boot_device_id' {" | sed "s/^/$submenu_indentation/"
      echo "menuentry '$(echo "$os" | grub_quote)' ${CLASS} \$menuentry_id_option 'gnulinux-simple-$boot_device_id' {" | sed "s/^/$submenu_indentation/"


In [53]:
# To other os's
sed -n "/--class os/p" /etc/grub.d/30_os-prober

menuentry '$(echo "${LONGNAME} $bitstr $onstr" | grub_quote)' --class osx --class darwin --class os \$menuentry_id_option 'osprober-xnu-$2-$(grub_get_device_id "${DEVICE}")'  {
menuentry '$(echo "${LONGNAME} $onstr" | grub_quote)' --class windows --class os \$menuentry_id_option 'osprober-chain-$(grub_get_device_id "${DEVICE}")' {
menuentry '$(echo "${LONGNAME} $onstr" | grub_quote)' --class windows --class os \$menuentry_id_option 'osprober-efi-$(grub_get_device_id "${DEVICE}")' {
menuentry '$(echo "$OS $onstr" | grub_quote)' --class gnu-linux --class gnu --class os \$menuentry_id_option 'osprober-gnulinux-simple-$boot_device_id' {
	menuentry '$(echo "$title" | grub_quote)' --class gnu-linux --class gnu --class os \$menuentry_id_option 'osprober-gnulinux-$LKERNEL-${recovery_params}-$boot_device_id' {
menuentry '$(echo "${LONGNAME} $onstr" | grub_quote)' --class hurd --class gnu --class os \$menuentry_id_option 'osprober-gnuhurd-/boot/gnumach.gz-false-$(grub_get_device_id "${DEVICE}")'

In [ ]:
sudo sed -i "s/--class os/--class os --users milad,karam" /etc/grub.d/30_os-prober

### Custom menu entries

- 06_custom -> First
- 15_custom -> After active

In [101]:
grep -Poz "menuentry 'Ubuntu' (.|\n)*?\}" /boot/grub/grub.cfg 

: 1

`menuentry 'Ubuntu' --class ubuntu --class gnu-linux --class gnu --class os {
    recordfail
    load_video
    gfxmode $linux_gfx_mode
    insmod gzio
    insmod part_msdos
    insmod ext2
    set root='hd0,msdos1'
    search --no-floppy --fs-uuid --set=root 3c52f-1134-1
    linux	/vmlinuz
    initrd	/initrd.img
}`

In [ ]:
# GRUB_DISABLE_OS_PROBER=true 

In [ ]:
sudo chmod -x /etc/grub.d/10_linux /etc/grub.d/20_linux_xen /etc/grub.d/30_os-prober

In [102]:
ll /vmlinuz

lrwxrwxrwx 1 root root 29 May  2 15:28 /vmlinuz -> boot/vmlinuz-4.4.0-77-generic


## Grub legacy

In [ ]:
sudo nano /boot/grub/menu.lst # mostly symlink to -> ./grub.conf

## Lilo

![lilo](img/bl/lilo.png)

In [ ]:
sudo nano /etc/lilo.conf

In [ ]:
sudo lilo

In [ ]:
man 5 lilo.conf

In [ ]:
man 8 lilo

- - -

### Resources:

https://www.gnu.org/software/grub/manual/grub.html  
https://help.ubuntu.com/community/Grub2  
https://help.ubuntu.com/community/Grub2/CustomMenus

- - -
## Lecture notes

- All lecture notes are available at github in `ipynb` format: [https://github.com/ravexina/linux-notes](https://github.com/ravexina/linux-notes). 


- There are `html` exports of project available at: [https://ravexina.github.io/linux-notes](https://ravexina.github.io/linux-notes).

## License
<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" />
</a>

<span property="dct:title">Linux Notes</span> by 
<a href="http://github.com/ravexina/Linux-Notes" property="cc:attributionName" rel="cc:attributionURL">Milad As (Ravexina)</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

- - -

<a rel="license" href="https://gitlab.com/ravexina/"><img alt="ravexina's gitlab" style="display:inline;border-width:0;height:30px;float:left;" src="img/logo/logo-square.png" /></a> 

<a rel="license" href="https://github.com/ravexina/"><img alt="ravexina's github" style="float:right;display:inline;border-width:0;height:30px;" src="img/logo/GitHub_Logo.png" /></a>